In [1]:
import numpy as np 
import scipy.stats as st
from scipy.stats import shapiro
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.stats.proportion import proportion_confint 

## Statistical hypothesis test

### 1) X ~ N(µ, $σ^2$)  --> Z test 

Uma máquina é programada para encher pacotes de café, com média µ e **variância** igual a **400g²**. Sabemos que a máquina está regulada se a quantidade média de gramas nos pacotes de café é de 
**µ = 500g**.  Para verificar se a máquina está regulada, colhemos uma amostra de **25** pacotes e verificamos se a produção está sob controle, isto é, se µ = 500g ou não. 

Determine se a máquina está regulada, considerando um **𝛼 = 0.01**.

**Ho: µ = 500g** 

**H1: µ $\neq$ 500g**

#### a) Dados, estatísticas e parametros

In [2]:
amostra_cafe = [501.58, 505.3, 497.62, 511.23, 502.13, 498.45, 496.38, 498.44, 498.45, 488.0, 508.1, 493.79, 505.24,
                492.62, 518.15, 488.77, 495.19, 510.56, 477.39, 508.69, 488.84, 503.39, 499.12, 511.97, 484.84]
var = 400 

#### b) Teste de normalidade 

In [3]:
alpha = 0.05 
stat_test, p_valor = shapiro(amostra_cafe) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados possuem dist. Normal')

Não rejeitamos a Hipótese Nula - Os dados possuem dist. Normal


#### c) Hypothesis testing

In [4]:
alpha = 0.01
nivel_confianca = 1 - alpha 
media_amostral = np.mean(amostra_cafe)
desvio_populacional = np.sqrt(var)
media_H0 = 500
n = len(amostra_cafe)

z_statistic = (media_amostral - media_H0)/(desvio_populacional/np.sqrt(n))
z_alpha = st.norm.ppf(alpha/2)

p_value = 2*norm.cdf(z_statistic)

if(p_value <  alpha):
    print("Rejeitamos a Hipótese Nula")
else:
    print("Não rejeitamos a Hipótese Nula")

print('-----------------------------------------------------------------------------------------')

IC_media = st.norm.interval(nivel_confianca, loc=media_amostral, scale=desvio_populacional/np.sqrt(n))

print("O intervalo de ", nivel_confianca*100 ,
      "% confiança é de: ", [round(num,2) for num in IC_media])

print('Estatística de teste:', round(z_statistic, 3))
print('Z crítico:', round(z_alpha, 3))       
print('Valor p:', round(p_value, 3))      

Não rejeitamos a Hipótese Nula
-----------------------------------------------------------------------------------------
O intervalo de  99.0 % confiança é de:  [489.07, 509.67]
Estatística de teste: -0.158
Z crítico: -2.576
Valor p: 0.875


### 2) X ~ N(?, ?)  --> T test + S estimator 

Um fabricante afirma que seus cigarros não contêm mais que 30 mg de nicotina. Uma amostra de 35 cigarros fornece média de 32.1 mg e desvio padrão de 2.2 mg. No nível de 5%, os dados refutam ou não a afirmação do fabricante?

**Ho: µ = 30g** 

**H1: µ > 30g**

#### a) Dados, estatísticas e parametros

In [5]:
amostra_cigarro =  [33.81, 31.9, 30.15, 27.55, 32.87, 36.74, 31.75, 30.06, 34.3, 33.09, 32.87, 32.25, 32.1, 31.91, 
                    31.13, 30.18, 33.0, 37.04, 34.62, 37.12, 33.3, 33.31, 30.91, 28.94, 33.51, 30.68, 31.99, 32.12,
                    31.64, 28.13, 29.21, 31.8, 32.52, 30.24, 30.57]

#### b) Teste de normalidade

In [6]:
stat_test, p_valor = shapiro(amostra_cigarro) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados possuem dist. Normal')

Não rejeitamos a Hipótese Nula - Os dados possuem dist. Normal


#### c) Hypothesis testing

In [7]:
alpha = 0.05
nivel_confianca = 1 - alpha 
media_amostral = np.mean(amostra_cigarro)
desvio_amostral = np.std(amostra_cigarro, ddof=1) 
media_H0 = 30
n = len(amostra_cigarro)
graus_de_liberdade = n - 1


t_statistic, p_value = st.ttest_1samp(a=amostra_cigarro, popmean=media_H0, alternative='greater')

if(p_value < alpha):
    print("Rejeitamos a Hipótese Nula")
else:
    print("Não rejeitamos a Hipótese Nula")
print('-----------------------------------------------------------------------------------------')

IC_media = st.t.interval(nivel_confianca, df = graus_de_liberdade,  loc=media_amostral, scale=desvio_amostral/np.sqrt(n))

print("O intervalo de ", nivel_confianca*100 ,
          "% confiança para a quantidade de nicotina nos cigarros é: ", 
      [round(num,2) for num in IC_media])

print('Estatística de teste:', round(t_statistic, 3))     
print('Valor p:', round(p_value, 3))

Rejeitamos a Hipótese Nula
-----------------------------------------------------------------------------------------
O intervalo de  95.0 % confiança para a quantidade de nicotina nos cigarros é:  [31.33, 32.86]
Estatística de teste: 5.534
Valor p: 0.0


### 3) X ~ B(n, ?)  --> Z test + p estimator 

Uma empresa desenvolveu uma nova vacina de  Covid, e afirma que a proporção de imunizados é maior do que 50%. Em uma amostra de 726 pessoas que tomaram a vacina, 668 estavam imunizadas. No nível de 5%, teste a afirmativa de que a proporção de imunizados é maior do que 50%.

**Ho: p = 0,50** 

**H1: p > 0,50**

#### a) Hypothesis testing

In [8]:
alpha = 0.05 
sucessos = 668
tentativas = 726
p0 = 0.5  

p_chapeu = round(sucessos / tentativas, 2)

z_statistic, p_value = sm.stats.proportions_ztest(sucessos, tentativas, value=p0, alternative='larger', prop_var=p0)

if(p_value <  alpha):
    print("Rejeitamos a Hipótese Nula")
else:
    print("Não rejeitamos a Hipótese Nula")
    
print('-----------------------------------------------------------------------------------------')

print('Estatística de teste:', round(z_statistic, 3))     
print('Valor p:', round(p_value, 3))


ICp_95 = proportion_confint(sucessos, tentativas, alpha=alpha)
print("O intervalo de ", (1-alpha)*100 ,
      "% confiança para a proporção de imunizados ser maior que 50% é : ", 
      [round(num,3) for num in ICp_95],"\n")

Rejeitamos a Hipótese Nula
-----------------------------------------------------------------------------------------
Estatística de teste: 22.639
Valor p: 0.0
O intervalo de  95.0 % confiança para a proporção de imunizados ser maior que 50% é :  [0.9, 0.94] 



### 4) X ~ N(?, ?)  --> Teste para variância de uma pop normal 

Na indústria farmacêutica, baixa variabilidade é sinônimo de qualidade. Para isso, constantemente monitora-se e corrige-se a produção de medicamentos para manter níveis aceitáveis de qualidade. Uma amostra de frascos de medicamento foi inspecionada em relação à concentração de um determinado princípio ativo na solução.

Sabe-se que o lote é rejeitado se claramente ultrapassar o limite de σ2 = 0.0009

Faça um teste para verificar se a variância é maior do que 0.0009, com α = 5%.

**Ho: σ = 0.0009** 

**H1: σ > 0.0009**

#### a) Dados, estatísticas e parametros

In [9]:
amostra_medicamento = [0.15,0.18,0.18,0.20,0.21,0.22,0.25,0.27,0.17,0.18,0.19,0.20,0.21,0.22,0.26,0.27]

#### b) Teste de normalidade

In [10]:
stat_test, p_valor = shapiro(amostra_medicamento) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados possuem dist. Normal')

Não rejeitamos a Hipótese Nula - Os dados possuem dist. Normal


#### c) Hypothesis testing

In [14]:
hipotese_alternativa = 'greater'
nivel_significancia = 0.05 
variancia_H0 = 0.0009
variancia_amostral = np.var(amostra_medicamento, ddof = 1).round(4) # variancia amostral
nivel_confianca = 1 - nivel_significancia
n = len(amostra_medicamento)
graus_de_liberdade = n - 1

qui_quadrado = ((n-1)*variancia_amostral) / variancia_H0

if(hipotese_alternativa == "two-sided"):
    p_value = 2 * (1 - st.chi2.cdf(qui_quadrado, graus_de_liberdade))
elif(hipotese_alternativa == "less"):
    p_value = st.chi2.cdf(qui_quadrado, graus_de_liberdade)
elif(hipotese_alternativa == "greater"):
    p_value = 1 - st.chi2.cdf(qui_quadrado, graus_de_liberdade)

if(p_value <  nivel_significancia):
    print("Rejeitamos a Hipótese Nula")
else:
    print("Não rejeitamos a Hipótese Nula") 
    
print('-----------------------------------------------------------------------------------------')

print('Estatística de teste:', round(qui_quadrado, 3))     
print('Valor p:', round(p_value, 3))
 
lower_bound = ((n-1) * variancia_amostral) / st.chi2.ppf(1 - (nivel_significancia/2), graus_de_liberdade)
upper_bound = ((n-1) * variancia_amostral) / st.chi2.ppf(nivel_significancia/2, graus_de_liberdade)

print(f"Intervalo 95% de Confiança da Variância: ({lower_bound.round(4)}, {upper_bound.round(4)})")

Não rejeitamos a Hipótese Nula
-----------------------------------------------------------------------------------------
Estatística de teste: 21.667
Valor p: 0.117
Intervalo 95% de Confiança da Variância: (0.0007, 0.0031)
